In [1]:
%matplotlib widget

import os
import glob
import json
import h5py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from uncertainties import unumpy

from SiPMStudio.processing.transforms import baseline_subtract

from SiPMStudio.analysis.dark import current_waveforms
from SiPMStudio.analysis.dark import integrate_current
from SiPMStudio.analysis.dark import spectrum_peaks
from SiPMStudio.analysis.dark import gain
from SiPMStudio.analysis.dark import wave_peaks
from SiPMStudio.analysis.dark import amplitudes
from SiPMStudio.analysis.dark import dark_count_rate
from SiPMStudio.analysis.dark import cross_talk_frac
from SiPMStudio.analysis.dark import afterpulsing_frac
from SiPMStudio.analysis.dark import waveform_find

In [2]:
def append_to_file(file, dataname, data):
    with h5py.File(file, "a") as h5_file:
        h5_file.create_dataset(dataname, data=data)
        
def overwrite_to_file(file, dataname, data):
    with h5py.File(file, "r+") as h5_file:
        del h5_file[dataname]
        h5_file.create_dataset(dataname, data=data)

In [3]:
path = "/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_cold1"
t2_path = os.path.join(path, "t2")
t2_files = glob.glob(t2_path+"/*.h5")

save_path = os.path.join(path, "plots")

In [4]:
t2_files

['/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_cold1/t2/t2_ketek_cold1_23.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_cold1/t2/t2_ketek_cold1_24.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_cold1/t2/t2_ketek_cold1_25.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_cold1/t2/t2_ketek_cold1_26.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_cold1/t2/t2_ketek_cold1_27.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_cold1/t2/t2_ketek_cold1_28.h5']

In [5]:
file_number = 5
sipm_file = h5py.File(t2_files[file_number], "r")
#print(sipm_file.keys())
waveforms = sipm_file["/processed/waveforms"][:]
times = sipm_file["/processed/timetag"][:] / 1000
bias = sipm_file["bias"][()]
sipm_file.close()

In [6]:
peaks, heights = wave_peaks(waveforms, height=0.50, distance=10)

100%|██████████| 334407/334407 [00:22<00:00, 14832.50it/s]


In [7]:
plt.figure()
plt.plot(waveforms[3])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
dark_rate = dark_count_rate(waveforms, times, peaks, heights, exclude=None, region=[4e4, 1e6], display=True)
dark_rate
plt.savefig(os.path.join(save_path, "dark_count_rate_"+str(bias)+".png"), dpi=600)

100%|██████████| 334407/334407 [00:03<00:00, 89765.45it/s]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
cross_talk_output = cross_talk_frac(heights)
cross_talk_output

100%|██████████| 334407/334407 [00:00<00:00, 925576.42it/s]


0.41583315380748676

In [10]:
new_peaks, new_heights = wave_peaks(waveforms, height=0.25, distance=5)

100%|██████████| 334407/334407 [00:23<00:00, 14384.72it/s]


In [11]:
after_pulse, t_rec = afterpulsing_frac(waveforms, new_peaks, new_heights, display=True, fit_range=[[25, 1600], [0.35, 0.99]])
print(after_pulse, t_rec)

100%|██████████| 334407/334407 [00:00<00:00, 657150.96it/s]
/usr/local/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

3.887478431970623e-05 0.831785576497535


In [12]:
finder_waveforms, finder_peaks, finder_heights = waveform_find(waveforms, new_peaks, new_heights, len(waveforms), [25, 1600], [0.35, 0.99])
after_pulse = len(finder_waveforms) / len(waveforms)
after_pulse

100%|██████████| 334407/334407 [00:00<00:00, 646778.26it/s]


3.887478431970623e-05

In [13]:
t_plot = np.linspace(0, 2*len(finder_waveforms[0]), len(finder_waveforms[0]))
plt.figure()
for wave in finder_waveforms:
    plt.plot(t_plot[0:800], wave[0:800], alpha=0.25)
plt.xlabel("Time (ns)")
plt.ylabel("P.E.")
plt.grid(alpha=0.25)
plt.savefig(os.path.join(save_path, "afterpulse_"+str(bias)+".png"), dpi=600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
append_to_file(t2_files[file_number], "dark_rate", [dark_rate[0], dark_rate[1]])
append_to_file(t2_files[file_number], "cross_talk", cross_talk_output)

In [ ]:
append_to_file(t2_files[file_number], "afterpulse", after_pulse)
append_to_file(t2_files[file_number], "t_rec", t_rec)

In [15]:
append_to_file(t2_files[file_number], "afterpulse", 0)
append_to_file(t2_files[file_number], "t_rec", 0)